In [1]:
import gdown

url = 'https://drive.google.com/uc?id=1LreozloM6RYSZ9tBNAv0acWB6Umb4NrX'

output = 'agatha_christie.zip'
gdown.download(url, output, quiet=False)

!unzip -q agatha_christie.zip

Downloading...
From: https://drive.google.com/uc?id=1LreozloM6RYSZ9tBNAv0acWB6Umb4NrX
To: /content/agatha_christie.zip
100%|██████████| 1.85M/1.85M [00:00<00:00, 195MB/s]


In [2]:
path = './agatha_christie/'

text_1 = open(path + 'assassinato_no_expresso_oriente.txt', 'r').read()
text_2 = open(path + 'contos_de_miss_marple.txt', 'r').read()
text_3 = open(path + 'e_nao_sobrou_nenhum.txt', 'r').read()
text_4 = open(path + 'e_no_final_a_morte.txt', 'r').read()
text_5 = open(path + 'morte_no_nilo.txt', 'r').read()
text_6 = open(path + 'nemesis.txt', 'r').read()
text_7 = open(path + 'o_assassinato_de_roger_ackroyd.txt', 'r').read()
text_8 = open(path + 'o_clube_das_tercas_feiras.txt', 'r').read()
text_9 = open(path + 'os_crimes_abc.txt', 'r').read()
text_10 = open(path + 'poirot_investiga.txt', 'r').read()
text_11 = open(path + 'retrato_inacabado.txt', 'r').read()
text_12 = open(path + 'um_corpo_na_biblioteca.txt', 'r').read()
text_13 = open(path + 'um_destino_ignorado.txt', 'r').read()
text_14 = open(path + 'uma_dose_mortal.txt', 'r').read()


text = text_1 + text_2 + text_3 + text_4 + text_5 + text_6 + text_7 + text_8 + text_9 + text_10 + text_11 + text_12 + text_13 + text_14

In [3]:
agatha_christie = open('agatha_christie.txt', 'w')
agatha_christie.write(text)
agatha_christie.close()

In [4]:
text = open('agatha_christie.txt', 'r').read()
vocab = sorted(set(text))

In [5]:
import numpy as np

char_to_ind = { char:ind for ind, char in enumerate(vocab) }
ind_to_char = np.array(vocab)
encoded_text = np.array([ char_to_ind[c] for c in text ])
seq_len = 120
total_num_seq = len(text)//(seq_len + 1)

In [6]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [7]:
import tensorflow as tf

char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len + 1, drop_remainder=True)

dataset = sequences.map(create_seq_targets)

batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [9]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [10]:
vocab_size = len(vocab)
embed_dim = 64
rnn_neurons = 1026

In [11]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim = embed_dim,
  rnn_neurons = rnn_neurons,
  batch_size = batch_size)

In [12]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

In [13]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [14]:
epochs = 30

In [15]:
model.fit(dataset, epochs=epochs)
model.save('agatha_christie_gen.h5')

Epoch 1/30
312/312 [==============================] - 45s 127ms/step - loss: 2.5964
Epoch 2/30
312/312 [==============================] - 41s 123ms/step - loss: 1.8625
Epoch 3/30
312/312 [==============================] - 42s 127ms/step - loss: 1.4970
Epoch 4/30
312/312 [==============================] - 40s 123ms/step - loss: 1.3388
Epoch 5/30
312/312 [==============================] - 40s 123ms/step - loss: 1.2615
Epoch 6/30
312/312 [==============================] - 42s 125ms/step - loss: 1.2126
Epoch 7/30
312/312 [==============================] - 43s 132ms/step - loss: 1.1776
Epoch 8/30
312/312 [==============================] - 42s 129ms/step - loss: 1.1498
Epoch 9/30
312/312 [==============================] - 43s 131ms/step - loss: 1.1273
Epoch 10/30
312/312 [==============================] - 41s 123ms/step - loss: 1.1079
Epoch 11/30
312/312 [==============================] - 42s 128ms/step - loss: 1.0904
Epoch 12/30
312/312 [==============================] - 43s 133ms/step - lo

In [16]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('agatha_christie_gen.h5')
model.build(tf.TensorShape([1, None]))

In [17]:
def generate_text(model, start_seed, gen_size=100, temp=1.0):
    num_generate = gen_size

    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = temp

    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [18]:
print(generate_text(model, "Poirot", gen_size=1000))

Poirot. — Acha que eu acredito nisso... Você quer dizer. Um homem muito agrestivo.
      — O quê? — dizia ela. — Mas se não está falando,
nha. A mancada de como eno marido, de modo e percepeu de ouvimonalidades postatos e, no meio da frente. Poirot não achou a mesma cigada grave, sob uma porta ágna — Penversieu que foi afirmando qualquer coisa — retrucou Poirot. — E você readman brilharam de Caroline a Bendida. Eu teria que quisem...

Ela embôlicatone, Piara apenas um sujeito removia com a perspectiva que entrou em condições de repente ele escrevera no papéis de Arden e enrolava-se aos quartos mais rixaim destruídas.

Ela veio até o motorista, um tipo de aspiros. Encontrou a recita de quem diria que ele daínca era inorente de Frank e dois corpo perigoso.



AII

			“‘Está tão absolutamente InIgntur a opinião pública era dura. Como sair, Jacqueline. Não percebia o tempo todo o tempo todo sabera.

Sua mente dele tentou amavam a mal. Mas Célia compraram gordo.

— Concordo com intenção, a 

In [19]:
print(generate_text(model, "Morte", gen_size=1000))

Morte conservado uma grande bela de um detalhe amigo Sir Henry dos Margueritos. Quando pudermos tomar nossas lucro tanto melancro disse que Sir Cuém a batir de todos o que Paton. Pode-se de mim. Quando me fiz. Dentro em voz alta, domingo poderia ter atirado em Londres e o coronel Bantry não está encarregado de concordar com M.

– São obsesve e tudo, em sua mãe no escritório de Dartor aos dois antigas antes morreram na aldeia. A mãe era meio comunista. Suas dois sempre fazem amíveixades ferroviária tendo sido alguém nunca recuso. Desoa devido ao acrasa para fora, estava passando de um olhar fixo de livro para casa. Quando já esperava, por sua vez, mas me assustou-se, pensou termonou uma companhia discreta, que confessou alguma coisa sobre a mente:

— Mary Dabem, monsieur Poirot — gouve o problema com tinta invadiável, mas três vezes junto com a polícia um dia fora útil nas entristez do mundo... precisamos explicá-lo do sr. Morley. Ele estivera esperando, teria a gorjeta de certo modo? N

In [20]:
print(generate_text(model, "Quarto Amarelo", gen_size=1000))

Quarto Amarelo que as menores seremos bem
verdadeiros, mas sem provas.

“É por surdo em seu letal, elas não reparei que entendamos a srta. Sainsbury Seale, que o senhor atendia fazer detalhes meus vitos reais da Inglaterra. Aírei irritada.

– A senhorita pudisse, teremos mais nada dele...

Poirot gentilze ansiosós real. No momento, eu dava meias rapidamente, quase que cuidadosa pela menina quase desconchad...

Ursula moço diretor da loja da Sra. Ascher, e amigos os anos, difícil dar. Peter me veio para Davenheim.

— Ah, isso, claro — disse BCulpes — limpou o gatela de costume, o inspetor, que ficava narrativa com a bandeja.

– Mas o senhor fica a resposta. – Mas é fácil eu estava. Já obviou, pensativa.

Poirot disse:

– Derrubou as mãos.

– Não mexericou a srta. Morley na mas de distância.

— Quero que a senhorita se lembra de teruo normal. Britânico na casa, confiante, discrettaram-se bruscamente, amigos para observá-lo de vez em quando. Estou corto de quem contratara o nome de Tim ho

In [21]:
print(generate_text(model, "Nilo", gen_size=1000))

Nilouxar outra loucura. Estava embolados de lã muro.

– Pois bem, que lado nome – Joce disse ao mar, Cornelea.

– Será que ele traz ao ar da minha missão?

O sr. Howard Raikes disse:

– A senhora a tinha para si mesmo. O piano era escuro, encontrou a matada por Miriam, em torno da moça — sobro conforme do meu país, além de ser uma coisa que por acaso assim o quí. Pobreviveis! Ele queria que o senhor pense.

Eles vão telefôncias, disseram que o sr. Jefferson teria, quero dizer. Os pais dela tenha se esgora natural. Esta é uma delicazendo de se sair. Tudo isso dirio que chamou Peter um vestido de libo. Lembre-se me disse que o detendesse baixando a voz inicial, sem necessito como a própria aventura. – E irritante do consultório? – indvolta – disse. – É uma coisa muito mais velha. Havia alguma costa quando essa sra. Sembra então neste lugar é preciso esperar uemos um jeito? Insistisse o deserto.





II


Tanto na vida era Majar Raymond Weslmer estar enganada, quando dirigentes. Mais a ma

In [22]:
print(generate_text(model, "Assassinato", gen_size=1000))


Assassinato, archur, e, no final, fizeram-na abafana tudo em você. Estou esperando um telegrama na direção de Ferro para cáleas diabóliciamos na sala. E agora me perguntei ao sol. Númera da sra vantagem de encantar a situação entreler vivo, e não deram. Somos importantes em nem cabo de amor setura. Disse:

– Vou dizer, até ontem, sentada nele, senhor. Mas Henet só posso dizer. Mademoiselle se pondo a pensa que Miss Jeanne não o atordoalidão está certo. Olhou assustado para trás, eno mundo. Bastava-se começou a traram-se sobre a mesa do café da manhã.

As lareneiras foram ali deitado, bem-moespero, insolente, até alfaberar. Dizia que início. Era isso. Como eu tracaram, diz que isso depende muito a carta como caridade o que quero dizer isso. Me ouviu falar de novo seus maridos, Lady Willard era aquela visita a deixar a ela e depois entraram. Tirou o dinheiro quando formou fogo.

– Não tinha formado vindo de carro azul no quiosque?"       Depois bateu profundamente malfeito. Então chegou 